In [7]:
# Import Dependencies
import billboard
import sqlite3
import requests
import json


In [8]:
# Call hot 100 tracks billbaord API using billboard library
chart = billboard.ChartData('hot-100')

In [9]:
artists = []
songs = []


for song in chart:
    
#     Add songs and artists to list for API calls
    songs.append(song.title)
    artists.append(song.artist)
# print(artists)
# print()
# print(songs)

In [10]:
# Call top 200 tracks billbaord API using billboard library
chart = billboard.ChartData('billboard-200')
# print(chart)

In [13]:
# Access token for Spotify API calls
# Enter your access_token given to you. Will import have to figure out a way to import from js

access_token = "BQCccsEqXhKa4g7gAhhiILFLnQYWZxE7wgnxJqCECHIJ0dca2voLMKVgdIWdYvnLSSffUatHW6_36kbhD7HGcLZhZMLeL6me0ewwRGV8X6I-a0DVu502mT0jK5UFBpvHooEdOFnCuY99kB3j6r376H6YuyO-CVUWrBPM"

In [14]:

top_songs_json = []
song_ids = []
popularity = []
markets = []

for song in songs:
    
#     Spotify API search endpoint returns list of tracks matching track title in order of popularity
    endpoint = f"https://api.spotify.com/v1/search?q={song}&type=track"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()
    
    try:
#         Apped first json object to list (The most popular track matching search criteria)
        track = response["tracks"]["items"][0]
        top_songs_json.append(track)
        
#         Append song_id for use in second API Call
        song_id = response["tracks"]["items"][0]["id"]
        song_ids.append(song_id)
        
        #  Append markets 
        market = response["tracks"]["items"][0]["available_markets"]
        markets.append(market)
        
    except: 
#         Alert for any track that was not appended to lists
        print("Song not Found")
        print(json.dumps(response["tracks"]["items"][0]["artists"],indent=4))


In [15]:
# Verify list length
print(len(song_ids))

100


In [16]:
# Count the number of countries in which each song is available
from collections import Counter
market_counter = Counter(x for sublist in markets for x in sublist)
# Counter(x for sublist in markets for x in sublist)
market_counter = dict(market_counter)
print(market_counter)

{'AD': 100, 'AR': 100, 'AT': 100, 'AU': 100, 'BE': 100, 'BG': 100, 'BO': 100, 'BR': 100, 'CA': 100, 'CH': 100, 'CL': 100, 'CO': 100, 'CR': 100, 'CY': 100, 'CZ': 100, 'DE': 100, 'DK': 100, 'DO': 100, 'EC': 100, 'EE': 100, 'ES': 100, 'FI': 100, 'FR': 100, 'GB': 100, 'GR': 100, 'GT': 100, 'HK': 100, 'HN': 100, 'HU': 100, 'ID': 100, 'IE': 100, 'IL': 100, 'IS': 100, 'IT': 100, 'JP': 100, 'LI': 100, 'LT': 100, 'LU': 100, 'LV': 100, 'MC': 100, 'MT': 100, 'MX': 100, 'MY': 100, 'NI': 100, 'NL': 100, 'NO': 100, 'NZ': 100, 'PA': 100, 'PE': 100, 'PH': 100, 'PL': 100, 'PT': 100, 'PY': 100, 'RO': 100, 'SE': 100, 'SG': 100, 'SK': 100, 'SV': 100, 'TH': 100, 'TR': 100, 'TW': 100, 'US': 100, 'UY': 100, 'VN': 100, 'ZA': 100}


In [25]:
market_dic = []

for key, value in market_counter.items():
    temp = {"country_code": key,
           "market_count": value}
    market_dic.append(temp)
    
print(market_dic)

[{'country_code': 'AD', 'market_count': 100}, {'country_code': 'AR', 'market_count': 100}, {'country_code': 'AT', 'market_count': 100}, {'country_code': 'AU', 'market_count': 100}, {'country_code': 'BE', 'market_count': 100}, {'country_code': 'BG', 'market_count': 100}, {'country_code': 'BO', 'market_count': 100}, {'country_code': 'BR', 'market_count': 100}, {'country_code': 'CA', 'market_count': 100}, {'country_code': 'CH', 'market_count': 100}, {'country_code': 'CL', 'market_count': 100}, {'country_code': 'CO', 'market_count': 100}, {'country_code': 'CR', 'market_count': 100}, {'country_code': 'CY', 'market_count': 100}, {'country_code': 'CZ', 'market_count': 100}, {'country_code': 'DE', 'market_count': 100}, {'country_code': 'DK', 'market_count': 100}, {'country_code': 'DO', 'market_count': 100}, {'country_code': 'EC', 'market_count': 100}, {'country_code': 'EE', 'market_count': 100}, {'country_code': 'ES', 'market_count': 100}, {'country_code': 'FI', 'market_count': 100}, {'country

In [ ]:
# raw json
audio_features = []

# List to Load into db
features = []

for song_id in song_ids:
    
#     Call Spotify API to collect Audio features data
    endpoint = f"https://api.spotify.com/v1/audio-features/{song_id}"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()
    
    song_id = response["id"]
    
    endpoint2 = f"https://api.spotify.com/v1/tracks/{song_id}"
    headers2 = {"Authorization":f"Bearer {access_token}"}

    response2 = requests.get(endpoint2,headers=headers2).json()
    
#     Append raw json to list
    audio_features.append(response)

    
#     Append each feature
    features.append({
        "id": response["id"],
        "popularity": response2["popularity"],
        "danceability": response["danceability"],
        "energy": response["energy"],
        "loudness": response["loudness"],
        "speechiness": response["speechiness"],
        "duration_ms": response["duration_ms"],
        "tempo": response["tempo"]
    })
    

In [ ]:
# print(features)
print(artists)

In [ ]:
genres = []

for artist in artists:
    
#     Spotify API search endpoint returns list of tracks matching track title in order of popularity
    endpoint = f"https://api.spotify.com/v1/search?q={artist}&type=artist"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()
    try:
        genre = response["artists"]["items"][0]
        genres.append(genre)
    except:
        print(f"Artist not found: {artist}")
    

In [ ]:
print(genres[1])

In [20]:
from sqlalchemy import create_engine

# Connect to db using sqlalchemy
db_uri = "sqlite:///music.db"
engine = create_engine(db_uri)

# create features table
# engine.execute('CREATE TABLE "features" ('
#                'id string NOT NULL,'
#                'popularity integer,'
#                'danceability integer,'
#                'energy integer,'
#                'loudness integer,'
#                'speechiness integer,'
#                'duration_ms integer,'
#                'tempo integer);')

# create map table
# engine.execute('CREATE TABLE "map" ('
#                'country_code string,'
#                'market_count integer);')


# c.execute("DROP TABLE features")


In [ ]:
conn = sqlite3.connect('music.db')

c = conn.cursor()


for feature in features:
    c.execute("insert into features values (?, ?, ?, ?, ?, ?, ?, ?)", 
              [feature['id'],feature['popularity'],feature['danceability'],feature['energy'],
              feature['loudness'],feature['speechiness'],feature['duration_ms'],feature['tempo']])
    conn.commit()
conn.close()

In [ ]:
# verify table creation
result = engine.execute('SELECT * FROM "features"')
# print(result.fetchall())

In [26]:

conn = sqlite3.connect('music.db')

c = conn.cursor()


for market in market_dic:
    c.execute("insert into map values (?, ?)", 
              [market['country_code'],market['market_count']])
    conn.commit()
conn.close()

In [27]:
# verify table creation
result = engine.execute('SELECT * FROM "map"')
print(result.fetchall())

[('AD', 100), ('AR', 100), ('AT', 100), ('AU', 100), ('BE', 100), ('BG', 100), ('BO', 100), ('BR', 100), ('CA', 100), ('CH', 100), ('CL', 100), ('CO', 100), ('CR', 100), ('CY', 100), ('CZ', 100), ('DE', 100), ('DK', 100), ('DO', 100), ('EC', 100), ('EE', 100), ('ES', 100), ('FI', 100), ('FR', 100), ('GB', 100), ('GR', 100), ('GT', 100), ('HK', 100), ('HN', 100), ('HU', 100), ('ID', 100), ('IE', 100), ('IL', 100), ('IS', 100), ('IT', 100), ('JP', 100), ('LI', 100), ('LT', 100), ('LU', 100), ('LV', 100), ('MC', 100), ('MT', 100), ('MX', 100), ('MY', 100), ('NI', 100), ('NL', 100), ('NO', 100), ('NZ', 100), ('PA', 100), ('PE', 100), ('PH', 100), ('PL', 100), ('PT', 100), ('PY', 100), ('RO', 100), ('SE', 100), ('SG', 100), ('SK', 100), ('SV', 100), ('TH', 100), ('TR', 100), ('TW', 100), ('US', 100), ('UY', 100), ('VN', 100), ('ZA', 100)]
